## c006 - 01/03/2017 - MOYA - Chemistry outflow on the West Coast of Africa

Flight c006 was part of the MOYA (Methane Observations and Yearly Assessments) campaign. On this sortie a transect was investigated by flying stacked runs over the Ocean on the West Coast of Africa. The flight track (see further down) shows that the aircraft spent most of the time between 13 degree and 12 degree North at the 16 degree meridian.

In [ ]:
%matplotlib inline

import cis
import datetime
import os

ROOT_DATA_PATH = '/home/axel/Dropbox/cis_workshop/data/c006'

os.chdir(ROOT_DATA_PATH)

### Merging CAMS data files into a single NetCDF

The data merging has already been done. The below code is left only for reference in case anybody wants to do it themself.

iris can do the merge for us. The only pitfall is that iris refuses to concatenate datasets with differing history attributes. Therefore this attribute needs to be removed (popped) from every individual dataset before the concatenate function is called.

In [ ]:
#import iris

## read all the data in
#cube_list = iris.load('z_cams_c_ecmf_20170301000000_prod_fc_pl*_aermr0*.nc')

## pop/remove the history attribute, otherwise merging is not possible
#for i in cube_list:
#    i.attributes.pop('history')

#ods = cube_list.concatenate()
## Use the compression option; this can reduce the size of your netCDF massively
#iris.save(ods, os.path.join(ROOT_DATA_PATH, 'moya_cams_c006.nc', zlib=True, packing='short'))

### Model data

The model data come from the Copernicus Atmospheric Monitoring Service (http://atmosphere.copernicus.eu/ftp-access-global-data). The available NetCDF covers the whole globe and at 25 pressure levels. The variables we use are the three different dust aerosols.

* aermr04	Dust Aerosol (0.03 - 0.55 um) Mixing Ratio
* aermr05	Dust Aerosol (0.55 - 0.9 um) Mixing Ratio
* aermr06	Dust Aerosol (0.9 - 20 um) Mixing Ratio

The current model products can be looked at online: http://atmosphere.copernicus.eu/catalogue#/

In [ ]:
cams_c006 = cis.read_data_list('moya_cams_c006.nc',
                               ['aermr04', 'aermr05', 'aermr06'])
print(cams_c006)

In [ ]:
# Let's look at a curtain
cams_curtain = cams_c006.subset(x=[16.5, 17.5],
                                y=[11,14],
                                z=[300, 1000])

In [ ]:
cams_curtain = cams_curtain.collapsed('time', how='mean')#
print(cams_curtain)

We like to see the relation of small to middle sized aerosol.

In [ ]:
# create a new GriddedDataList
grid_list = cis.data_io.gridded_data.GriddedDataList()
grid_list.append(cams_curtain[0])
grid_list.append(cams_curtain[1])
ax = grid_list.plot(how='comparativescatter')
ax.grid()

In [ ]:
cams_curtain = cams_curtain.collapsed('longitude', how='mean')

In [ ]:
# TODO: That does not work :(
cams_curtain[0].plot(xaxis='latitude')

In [ ]:
# TODO: Why can I not plot like this
cams_curtain.plot()

In [ ]:
cams_curtain[0].data.shape

In [ ]:
p = cams_curtain[0]

In [ ]:
# TODO: I can plot a curtain like this, but I would like CIS to do it for me
imshow(p.data)

### Aircraft data

Read in the total scatter and back scatter data from the Nephelometer.

In [ ]:
ara = cis.read_data_list('core_faam_20170301_v004_r0_c006_1hz_edited.nc', \
                         ['TSC_BLUU', 'TSC_GRNU', 'TSC_REDU', 'BSC_BLUU', 'BSC_GRNU', 'BSC_REDU', 'PS_RVSM'],
                         product='NCAR_NetCDF_RAF')

In [ ]:
print(ara)

In [ ]:
# TODO: Make the variables share one colorbar
ax = ara.plot()
ax.grid(True)

In [ ]:
# TODO: Would be nice to compare a model profile with a profile from the FAAM aircraft.

Add pressure as a new coordinate to the CIS aircraft dataset.

In [ ]:
from cis.data_io.Coord import Coord
from cis.data_io.ungridded_data import Metadata, UngriddedData, UngriddedDataList

# Read in air pressure as a new variable
pressure_data = cis.read_data('core_faam_20170301_v004_r0_c006_1hz_edited.nc', 'PS_RVSM')

pres = Coord(ara[6].data.ravel().data, Metadata('air_pressure', units='millibars'), 'p')

In [ ]:
ara2 = cis.data_io.ungridded_data.UngriddedDataList()

coord_list = [ara.coord('latitude'),
              ara.coord('longitude'),
              pres,
              ara.coord('altitude'),
              ara.coord('time')]

for i in ara:
    ara2.append(UngriddedData(i.data.ravel(),
                              i.metadata,
                              coord_list))

In [ ]:
print(ara2)

In [ ]:
print(cams_c006[0])

In [ ]:
ara.coord('latitude')